In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import glob
import os
import numpy as np
from PIL import Image
import torchvision.models as models
import copy
from torchvision.utils import save_image
import PIL
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from lib import data_loader as dl
from lib import models as md
from lib import model_device_io as io
from torch.autograd import Variable

In [2]:
train_set = dl.OuluNpu('data/oulu_npu_cropped/train')
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=0)
test_set = dl.OuluNpu('data/oulu_npu_cropped/val')
test_loader = DataLoader(train_set, batch_size=512, shuffle=False, num_workers=0)

In [4]:
device = io.GetCudaDevice(cuda = 1, seed = 123, log=True)
model = md.ResNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.5, 0.999))
#io.loadModel('models/best.pth',model,optimizer)

Device used: cuda:1


In [5]:
def test():
    model.eval()
    loss, correct = 0, 0
    with torch.no_grad():
        for idx, datas in enumerate(test_loader):
            imgList, phone, session, human_id, access, _ = datas
            imgList = [img.to(device) for img in imgList]
            access = access.to(device)
            output = model(imgList)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(access.view_as(pred)).sum().item()
            loss += F.cross_entropy(output, access).item()*len(access)
                
    loss /= len(test_loader.dataset)
    percentage = 100.*correct/len(test_loader.dataset)
    print('***Validation Result:\tLoss: {:.4f}\tAccuraccy: {:.2f}% ({}/{})\n'.format(loss, percentage,
          correct, len(test_loader.dataset)))
    return percentage

def train(train_epoch = 100):
    best_acc = 0
    for epoch in range(train_epoch):
        model.train()
        for batch_idx, datas in enumerate(train_loader):
            optimizer.zero_grad()
            imgList, phone, session, human_id, access, _ = datas
            imgList = [img.to(device) for img in imgList]
            access = access.to(device)
                
            #train
            output = model(imgList)
            loss = F.cross_entropy(output, access)
            loss.backward()
            optimizer.step()
            
            #log    
            if (batch_idx+1) % max(1, int(len(train_loader)/5)) == 0:
                print('Training Epoch: {}/{}\tBatch: {}/{}\tLoss: {:.4f}'.format(epoch+1,
                       train_epoch, batch_idx+1, len(train_loader), loss.item()))
        acc = test()
        if acc > best_acc:
            best_acc = acc
            io.saveModel('models/best.pth', model, optimizer)
            

In [ ]:
train()

Training Epoch: 1/100	Batch: 1/5	Loss: 1.6142
Training Epoch: 1/100	Batch: 2/5	Loss: 1.7630
Training Epoch: 1/100	Batch: 3/5	Loss: 1.5545
Training Epoch: 1/100	Batch: 4/5	Loss: 1.4256
Training Epoch: 1/100	Batch: 5/5	Loss: 1.3488
***Validation Result:	Loss: 1.2203	Accuraccy: 46.92% (563/1200)

model saved to models/best.pth
Training Epoch: 2/100	Batch: 1/5	Loss: 1.2379
Training Epoch: 2/100	Batch: 2/5	Loss: 1.0392
Training Epoch: 2/100	Batch: 3/5	Loss: 0.9899
Training Epoch: 2/100	Batch: 4/5	Loss: 0.9508
Training Epoch: 2/100	Batch: 5/5	Loss: 0.8748
***Validation Result:	Loss: 0.6537	Accuraccy: 69.08% (829/1200)

model saved to models/best.pth
Training Epoch: 3/100	Batch: 1/5	Loss: 0.7082
Training Epoch: 3/100	Batch: 2/5	Loss: 0.7526
Training Epoch: 3/100	Batch: 3/5	Loss: 0.7319
Training Epoch: 3/100	Batch: 4/5	Loss: 0.5583
Training Epoch: 3/100	Batch: 5/5	Loss: 0.5299
***Validation Result:	Loss: 0.4061	Accuraccy: 80.33% (964/1200)

model saved to models/best.pth
Training Epoch: 4/100	